In [2]:
import pandas as pd

In [11]:
train = pd.read_pickle('../input/tweet-sentiment-extraction/preds.pkl')

In [23]:
def get_pos(x, y):
    return x.find(y)

def jaccard_list(l1, l2):
    a = set(l1)
    b = set(l2)
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def get_jaccard(x, y):
#     label = x['selected_text'].split()
#     label2 = x['pred'].split()
    return jaccard_list(x.split(), y.split())

def broken_start(x, y):
    
    if y>0 and x[y-1].isalpha():
        return True
    return False

def broken_end(x, y):
    if y<len(x) and x[y]!=' ':
        return True
    return False

In [24]:
train['jaccard'] = train.apply(lambda x: get_jaccard(x['selected_text'], x['pred']), axis=1)

In [64]:
train[train['jaccard']<0.2][['text','sentiment','selected_text','pred','new_st']].sample(n=5)

,text,sentiment,selected_text,pred,new_st
17972,Can`t beat all time low.. (: I soooooo want to...,negative,Can`t beat,Can`t beat all time low.. (: I soooooo want to...,Can`t beat
5427,(blinks fast) its better now ... Lol thanks :-*,positive,thanks,its better now ... Lol thanks,thanks
2205,Hey Honey Bunny here big bunny hugs,positive,unny,big bunny hugs,her
18546,morning darlin` Hope you`re feelin` tip-top ...,positive,morning darlin` Hope you`re feelin` tip-top,Hope,morning darlin` Hope you`re feelin` tip-top
25826,Go Canada they made Nikkie Payne #1 comic well...,positive,Go Canada they made Nikkie Payne #1 comic well...,funny,Go Canada they made Nikkie Payne #1 comic well...


In [17]:
train['clean_text'] = train['text'].apply(lambda x: ' '.join(x.strip().split()))
train['clean_st'] = train['selected_text'].apply(lambda x: ' '.join(x.strip().split()))
train['start_pos_origin'] = train.apply(lambda x: get_pos(x['text'], x['selected_text']), axis=1)
train['end_pos_origin'] = train['start_pos_origin']+train['selected_text'].str.len()
train['start_pos_clean'] = train.apply(lambda x: get_pos(x['clean_text'], x['clean_st']), axis=1)
train['end_pos_clean'] = train['start_pos_clean']+train['clean_st'].str.len()

In [20]:
train['broken_start'] = train.apply(lambda x: broken_start(x['clean_text'], x['start_pos_clean']), axis=1)
train['broken_end'] = train.apply(lambda x: broken_end(x['clean_text'], x['end_pos_clean']), axis=1)

In [21]:
def get_clean_label(x):
    if x['start_pos_clean']==0 or not x['broken_start']:
        return x['clean_st']
    clean_text = x['clean_text']
    start = x['start_pos_origin']
    length = len(' '.join(x['selected_text'].split()))
    new_st = clean_text[start:start+length]
    # delete last char 
    new_st_words = new_st.split()
    if len(new_st_words)>1 and len(new_st_words[-1])==1 and start+length<len(clean_text) and clean_text[start+length].isalpha():
#         print(x['clean_st'])
#         print(new_st)
        new_st = ' '.join(new_st_words[:-1])
#         print(new_st)
#         print('-'*20)
    return new_st

In [22]:
train['new_st'] = train.apply(lambda x: get_clean_label(x), axis=1)

In [25]:
train['jaccard2'] = train.apply(lambda x: get_jaccard(x['new_st'], x['pred']), axis=1)

In [27]:
train['jaccard'].mean()

0.714660893295629

In [28]:
train['jaccard2'].mean()

0.723684122394082

In [49]:
def post_processing(x):
    start = x['clean_text'].find(x['pred'])
    length = len(x['pred'])
    if len(x['pred'])/len(x['clean_text'])>0.9:
        return x['pred']
    shift = max(0, len(x['text'])-len(x['clean_text'])-1)
    return x['clean_text'][start-shift:start+length-shift]

In [50]:
train['pred2'] = train.apply(lambda x: post_processing(x), axis=1)

In [51]:
train[['text','pred','pred2']].head()

,text,pred,pred2
0,i`m so bored i can barely even tweet. i have n...,bored,bored
1,awwwwwwwwww thats jus...awwwww.did she get t...,awwwwwwwwww thats jus...awwwww.did she get to ...,awwwwwwwwww thats jus...awwwww.did she get to ...
2,I did not twitt yesterday cause it was a very ...,I can not sleep,I can not slee
3,Congratulation`s to phil packer on completing ...,Congratulation`s,Congratulation`s
4,O`Charleys? Pretty good. Especially when its f...,Pretty good.,Pretty good.


In [52]:
train['jaccard3'] = train.apply(lambda x: get_jaccard(x['selected_text'], x['pred2']), axis=1)

In [53]:
train['jaccard3'].mean()

0.6752573494814437